In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import copy
import os
from tqdm import tqdm

In [12]:
# Load and save image function

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_image(path, target_size = None):
    image = Image.open(path).convert('RGB')
    
    if target_size is not None:
        # Image.LANCZOS is a high-quality down-sampling filter 
        # (also known as "Lanczos resampling")
        # It preserves detail better than simpler filters like NEAREST, BILINEAR
        image = image.resize((target_size, target_size), Image.LANCZOS)

    # Pixel values from range 0-255 to range 0.0-0.1
    # axes are reordered from Pillow's (H,W,C) to (C,H,W)
    # which is the Pytorch expected
    loader = transforms.Compose([transforms.ToTensor()])

    # inserts a new dimension at index 0,
    # turning the tensor into (1,C,H,W)
    # the leading 1 represents the batch size - 
    # even if you load a single image, most PyTorch models expect a batch dimension
    tensor = loader(image).unsqueeze(0) # shape: (1, C, H, W)
    return tensor.to(device, torch.float)

def save_image(tensor, path):
    tensor = tensor.detach().cpu().squeeze(0).clamp(0,1)
    unloader = transforms.ToPILImage()
    image = unloader(tensor)
    image.save(path)

In [ ]:
path = "Cezanne/Cezanne1.jpg"

tensorA = load_image(path)

In [19]:
# VGG & feature extraction 

# load VGG-19 with the canonical ImageNet weights
vgg_weights = models.VGG19_Weights.IMAGENET1K_V1

# with the feature we keep only the convolutional and pooling layers
# the eval is used to set the model in inference mode
# meaning that layers like dropout or batchnorm will work in inference mode 
# and they will not introduce randomness in the computations
cnn = models.vgg19(weights = vgg_weights).features.to(device).eval()

In [20]:
print(models.VGG19_Weights.__members__)   # shows all available weight keys

{'IMAGENET1K_V1': VGG19_Weights.IMAGENET1K_V1, 'DEFAULT': VGG19_Weights.IMAGENET1K_V1}


In [ ]:

# Normalization used by the pretrained VGG models
# RGB values for the training set
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std  = torch.tensor([0.229, 0.224, 0.225]).to(device)

In [ ]:
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super().__init__()
        # reshape to [C x 1 x 1] so it can be broadcasted
        self.mean = mean.clone().detach().view(-1,1,1)
        self.std  = std.clone().detach().view(-1,1,1)
    def forward(self, img):
        return (img - self.mean) / self.std
